<a href="https://colab.research.google.com/github/kkrbalam/Python/blob/master/blazingsql_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get Started with Blazing SQL

In this notebook, we will walk through the process for getting BlazingSQL and cuDF running. We will then demonstrate a basic ETL process and query a csv. 

Please click the 'Open in Playground Mode Option ' to run this demo on your own. 

Feel free to add your own code or modify ours as needed.


Background Info:
[BlazingSQL](http://blazingdb.com/#/)
[RAPIDS AI/cuDF](https://rapids.ai/about.html)


# Setup

## Environment Sanity Check 

Once you are in Playground mode, click the Runtime dropdown at the top of the page, then Change Runtime Type and confirm the instance type has GPU selected for 'Hardware Accelerator'.

Check the output of '!nvidia-smi' to make sure you've been allocated a Tesla T4.


In [0]:
!nvidia-smi

Thu Jul 18 00:10:37 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    31W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Sometimes Google Colab will assign a Tesla K80 (which we do not support), so the script below helps confirm you have a T4 GPU attached to your runtime.

In [0]:
import pynvml


pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)

if device_name != b'Tesla T4':
  raise Exception("""
    Unfortunately this instance does not have a T4 GPU.
    
    Please make sure you've configured Colab to request a GPU instance type.
    
    Sometimes Colab allocates a Tesla K80 instead of a T4. Resetting the instance.

    If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
  print('Woo! You got the right kind of GPU!')

Woo! You got the right kind of GPU!


## Install BlazingSQL + cuDF
This will take roughly a minute and a half.

cuDF is the RAPIDS AI GPU DataFrame library with a Pandas-like API. Learn more about cuDF [here](https://github.com/rapidsai/cudf).

In [0]:
!bash -c "$(wget -q https://s3.amazonaws.com/blazingsql-colab/install.sh -O -)"

PYTHON: python3
VERSION: Python 3.6.8
PIP: pip3
VERBOSE: /dev/null
SUDO: 
PATH_USR: /usr/
### update ###
### dependencies ###
(Reading database ... 131779 files and directories currently installed.)
Removing cmake (3.10.2-1ubuntu2) ...
Removing google-perftools (2.5-2.2ubuntu3) ...
Removing curl (7.58.0-2ubuntu3.7) ...
Removing r-cran-devtools (2.0.2-1cran1ppabionic0) ...
Removing r-cran-usethis (1.5.0-1cran1ppabionic0) ...
Removing r-cran-tidyverse (1.2.1-3cran1ppa0bionic0) ...
Removing r-cran-modelr (0.1.4-1cran1ppabionic0) ...
Removing r-base (3.6.0-2bionic) ...
Removing r-recommended (3.6.0-2bionic) ...
Removing r-cran-boot (1.3-22-1cran1ppabionic0) ...
Removing libcurl4-openssl-dev:amd64 (7.58.0-2ubuntu3.7) ...
Removing r-cran-readxl (1.3.1-1cran1ppabionic0) ...
Removing r-cran-haven (2.1.0-1cran1ppabionic0) ...
Removing littler (0.3.8-1bionic0) ...
Removing r-base-dev (3.6.0-2bionic) ...
Removing r-cran-rvest (0.3.4-1cran1ppabionic0) ...
Removing r-cran-progress (1.2.2-1cran1ppab

## Sanity Check
Confirm all three BlazingSQL components are running. These are the main components of the BlazingSQL engine and will confirm that the install process worked.

In [0]:
!blazingsql status

blazing-calcite                  RUNNING   pid 3202, uptime 0:00:02
blazing-orchestrator             RUNNING   pid 3204, uptime 0:00:02
blazing-ral                      RUNNING   pid 3203, uptime 0:00:02


## Clear Runtime

Before you continue, please go to the 'Runtime' Menu above, and select 'Reset Runtime'. 

# You are ready to go with BlazingSQL!
Nice Job! Now lets see how it works.

# Import packages and create Blazing Context
You can think of the BlazingContext much like a Spark Context. This is where information such as FileSystems you have registered, Tables you have created will be stored.

If you have issues running this cell, restart runtime and try running it again.


In [0]:
# Set Environment Variables
import sys, os
os.environ["NUMBAPRO_NVVM"] = "/usr/local/cuda/nvvm/lib64/libnvvm.so"
os.environ["NUMBAPRO_LIBDEVICE"] = "/usr/local/cuda/nvvm/libdevice/"

# Import RAPIDS AI stack
from blazingsql import BlazingContext
import cudf

bc = BlazingContext()

connection established


# Read CSV
First we uploaded a CSV file through the Google Colab interface. 

Then we use cuDF to read the CSV file. This gives us a GPU DataFrame (GDF).

To learn more about the GDF and how it enables end to end workloads on rapids, read [this](https://blog.blazingdb.com/blazingsql-part-1-the-gpu-dataframe-gdf-and-cudf-in-rapids-ai-96ec15102240).

In [0]:
#Download the test CSV
!wget 'https://s3.amazonaws.com/blazingsql-colab/Music.csv'

--2019-07-18 00:14:40--  https://s3.amazonaws.com/blazingsql-colab/Music.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.161.133
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.161.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10473 (10K) [text/csv]
Saving to: ‘Music.csv’

Music.csv           100%[===================>]  10.23K  --.-KB/s    in 0s      

2019-07-18 00:14:40 (179 MB/s) - ‘Music.csv’ saved [10473/10473]



In [0]:
gdf = cudf.read_csv('Music.csv')


# Create a Table

Now we just need to create a table. Apache Parquet is a great, open source and distributed file format built for systems like HDFS. Apache Parquet files also have metadata which self describes the schema, making import a cinch!

In [0]:
bc.create_table('music', gdf)

# Query a Table
That's it! Now you can write a SQL query and the data will get processed on the GPU with BlazingSQL, and the output will be a GPU DataFrame (GDF) inside RAPIDS!

In [0]:
#Query
result = bc.sql('SELECT * FROM main.music').get()

#Get GDF
result_gdf = result.columns

#Print GDF
print(result_gdf)

                         ARTIST  RATING    YEAR       LOCATION  FESTIVAL_SET
0                  Arcade Fire    10.0  2018.0      Las Vegas           1.0
1                      Justice    10.0  2018.0      Las Vegas           1.0
2     Florence and The Machine    10.0  2018.0      Las Vegas           1.0
3                       Odesza    10.0  2018.0          Indio           1.0
4                     Bon Iver    10.0  2017.0          Indio           1.0
5  LA Philharmonic + Sigur Ros    10.0  2017.0             LA           0.0
6                    Sigur Ros    10.0  2014.0          Malmo           0.0
7                  Arcade Fire    10.0  2014.0          Indio           1.0
8                       Escort     9.0  2018.0  San Francisco           0.0
9                      Phoenix     9.0  2018.0       Berkeley           0.0
[336 more rows]


And... thats it! You are now live with BlazingSQL. Check out our [docs](https://docs.blazingdb.com) to get fancy as well as to learn more about how BlazingSQL works with the rest of [RAPIDS AI](https://rapids.ai/). 